In [ ]:
from functools import reduce
def init(buffer):
   # create spatial variables of specified number
   temp = buffer.pop(0)
   Vars = int(temp.split()[0].strip())
   TypeId = temp.split()[1].strip()
   
   # initialize constraints
   Id = 128
  
   ConMatrix = [None for v in range(Vars)]
   for v in range(Vars):
      j = {}   
      j[v] = Id
      ConMatrix[v] = j

   # parse spatial CSP
   for line in buffer:
        l = line.strip().replace('(','').replace(')','').split()
        if l == ['.']:
            break
        ConMatrix[int(l[0])][int(l[1])] = 255
        ConMatrix[int(l[1])][int(l[0])] = 255
   
   return TypeId, ConMatrix
def MCS(ConMatrix, neighbors):
    
   a = [None for i in range(len(ConMatrix))]
   a_ = [None for i in range(len(ConMatrix))]     

   Set = [None for i in range(len(ConMatrix))]
   Size = [None for i in range(len(ConMatrix))] 

   for i in range(len(ConMatrix)):
      Set[i] = set([])     
      Size[i] = 0
      Set[0].add(i)

   j = 0
   for i in range(len(ConMatrix)):
      v = Set[j].pop()
      a[v] = i
      a_[i] = v
      # # uncomment codes below to fix the bug
      # if j==(len(ConMatrix)-1):
      #    break
      Size[v] = -1
      for u in neighbors[v]:
         if Size[u] >= 0:
            Set[Size[u]].remove(u)
            Size[u] = Size[u] + 1
            Set[Size[u]].add(u)
      j = j + 1
      while j >= 0 and not Set[j]:
         j = j - 1

   return a, a_

def FIC(ConMatrix, neighbors, pair ):
   a = pair[0]
   a_ = pair[1]
   f = [None for i in range(len(ConMatrix))]
   index = [None for i in range(len(ConMatrix))]
   fill = list()

   for i in range(len(ConMatrix)-1,-1,-1):
      w = a_[i]
      f[w] = w
      index[w] = i
      for v in neighbors[w]:
         if a[v] > i:
            x = v
            while index[x] > i:
               index[x] = i
               fill.append((x,w) if x<w else (w,x))
               x = f[x]
            if f[x] == x: f[x] = w

   return fill


In [ ]:
buffer = [
    "4 #csp",
    "0 1 ( PO )",
    "0 2 ( PO )",
    "0 3 ( PO )",
    "1 2 ( PO )",
    "1 3 ( PO )",
    "2 3 ( PO )"
    "."
]

In [ ]:
TypeId, ConMatrix = init(buffer)
buffer = []
edjes = set([])
neighbors = tuple([set([]) for i in range(len(ConMatrix))])
for n, i in enumerate(ConMatrix):
    for j in i:
        if j > n:
            edjes.add((n,j))
            neighbors[n].add(j)
            neighbors[j].add(n)

# variable ordering computation with MCS
a,a_ = MCS(ConMatrix, neighbors)

fill = FIC(ConMatrix, neighbors, (a,a_))